In [69]:
# import necessary libraries
import numpy as np
import argparse
import imutils
import cv2
import pandas as pd
import zipfile
from google.colab import files
import io
import os
import zipfile
from PIL import ImageDraw,Image,ImageFont
from google.colab.patches import cv2_imshow

In [70]:
zip_file = '/content/pawn_dataset.zip'
zip_ref = zipfile.ZipFile(zip_file,'r')
zip_ref.extractall('/content')
zip_ref.close()

In [71]:
# annotations = pd.read_csv('/content/pawn_dataset.csv')

In [72]:
# annotations.head

In [73]:
import tensorflow as tf

In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [75]:
chess_data = os.path.join('/content/content/pawn_dataset')
data = ImageDataGenerator(validation_split=0.2,rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [76]:
# index = 0
# for i, row in annotations.iterrows():
#   pic_name = row['Name']
#   image = cv2.imread('/content/content/data/pawn_classification_dataset/'+pic_name)
#   if(row['Label']=="pawn"):
#     cv2.imwrite("/content/pawn_dataset/pawn/pic{}".format(str(index)+".jpg"),image)
#   else:
#     cv2.imwrite("/content/pawn_dataset/not pawn/pic{}".format(str(index)+".jpg"),image)
#   index = index+1

In [77]:
pawn_train_dir = data.flow_from_directory(chess_data,subset='training',batch_size = 20,class_mode='binary',target_size=(150,150))
pawn_valid_dir = data.flow_from_directory(chess_data,subset='validation',batch_size = 20,class_mode='binary',target_size=(150,150))

Found 2297 images belonging to 2 classes.
Found 573 images belonging to 2 classes.


In [78]:
x,y=pawn_train_dir.next()
x.shape

(20, 150, 150, 3)

In [79]:
# from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape = (224,224,3),padding='same'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(2,2),

#     tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(2,2),

#     tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.GlobalMaxPool2D(),

    
#     # tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024,activation='relu'),
#     # tf.keras.layers.Dropout(0.5),
#     # tf.keras.layers.Dense(512,activation='relu'),
#     # tf.keras.layers.Dense(256,activation='relu'),
#     tf.keras.layers.Dense(1,activation='sigmoid')
# ]
# )
# model.summary()

In [80]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [81]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
# model.compile(optimizer=RMSprop(lr=0.01),loss='binary_crossentropy',metrics=['acc'])
model.compile(optimizer=SGD(lr=0.01),loss='binary_crossentropy',metrics=['acc'])

In [82]:
history = model.fit_generator(epochs=10,generator=pawn_train_dir,validation_data=pawn_valid_dir,verbose=1)

Epoch 1/10
115/115 [==============================] - 15s 130ms/step - loss: 0.7790 - acc: 0.5903 - val_loss: 1.8925 - val_acc: 0.5480
Epoch 2/10
115/115 [==============================] - 15s 129ms/step - loss: 0.5689 - acc: 0.6922 - val_loss: 2.9377 - val_acc: 0.5480
Epoch 3/10
115/115 [==============================] - 15s 130ms/step - loss: 0.5093 - acc: 0.7462 - val_loss: 1.3878 - val_acc: 0.5480
Epoch 4/10
115/115 [==============================] - 15s 129ms/step - loss: 0.4550 - acc: 0.7819 - val_loss: 0.5307 - val_acc: 0.7243
Epoch 5/10
115/115 [==============================] - 15s 128ms/step - loss: 0.4344 - acc: 0.7997 - val_loss: 0.4527 - val_acc: 0.7714
Epoch 6/10
115/115 [==============================] - 15s 128ms/step - loss: 0.3866 - acc: 0.8293 - val_loss: 0.4049 - val_acc: 0.8063
Epoch 7/10
115/115 [==============================] - 15s 129ms/step - loss: 0.3713 - acc: 0.8341 - val_loss: 0.3689 - val_acc: 0.8394
Epoch 8/10
115/115 [==============================] - 1

In [86]:
# from matplotlib import pyplot as plt
# plt.imshow(history['val_loss'])